In [3]:
import galois
import numpy as np
import math
# from py_ecc.bn128 import curve_order

curve_order = 967

# Creating a finite field (Galois field) which is basically Fq where q = 967
GF967 = galois.GF(curve_order)
# print(GF.properties)

######################################

def process_array(array):
    (rows, columns) = array.shape
    for i in range(0, rows):
        for j in range(0, columns):
            if(math.modf(array[i][j])[0] == 0.0):
                if(array[i][j] < 0):
                    array[i][j] = GF967(0) - GF967(-(array[i][j]))
            else:
                raise Exception("Fractional values not allowed in the R1CS arrays over prime field", GF967)
    return GF967(array)

######################################

lagrange_x_values = GF967(np.array([1,2,3]))

# These are the default R1CS values of L, R, S and W.
# Later we can take user inputs to create custom R1CS
default_L = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,4]
]))

default_R = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0]
]))

S = np.array([
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [2,1,0,0,-1,0]
])

default_S = process_array(S)

default_W = GF967(np.array([1,199,3,4,9,16]))

###############################################

def reverse_tuple(tuple):
    new_tuple = tuple[::-1]
    return new_tuple

def find_num_columns_in_array(arr):
    shape_tuple = arr.shape
    if(len(shape_tuple) == 2): 
        return shape_tuple[1]
    elif (len(shape_tuple) == 1): 
        return shape_tuple[0]
    else:
        raise Exception("Invalid array passed")
    
def find_num_points_for_lagrange(arr):
    shape_tuple = arr.shape
    return shape_tuple[0]

def convert_column_to_lagrange_polynomial(array, col_num: int):
    required_column = array[ :, col_num: (col_num + 1)].flatten()
    col_polynomial = galois.lagrange_poly(lagrange_x_values, required_column)
    
    if(col_polynomial == 0):
        col_polynomial_array = GF967(np.zeros(find_num_points_for_lagrange(array), dtype = int))
    elif(col_polynomial.coeffs.size < find_num_points_for_lagrange(array)):
        num_zeros_required = find_num_points_for_lagrange(array) - np.asarray(col_polynomial.coeffs).shape[0]
        col_polynomial_array = GF967(np.append(np.zeros(num_zeros_required, dtype=int), (np.asarray(col_polynomial.coeffs))))
    else:
        col_polynomial_array = GF967(np.asarray(col_polynomial.coeffs))
    
    return col_polynomial_array

def convert_arrays_into_poly_arrays(array):
    num_columns = find_num_columns_in_array(array)
    for i in range(num_columns):
        if(i == 0):
            first_arr = convert_column_to_lagrange_polynomial(array, 0)
            poly_arrays = GF967(np.array([first_arr]))
        else:
            poly_arr = convert_column_to_lagrange_polynomial(array, i)
            poly_arrays = GF967(np.append(poly_arrays, poly_arr))

    required_poly_array_shape = reverse_tuple(array.shape)
    return GF967(poly_arrays.reshape(required_poly_array_shape).transpose())

def multiply_poly_arrays_with_witness(array):
    return np.matmul(array, default_W)

# ###############################################

U = convert_arrays_into_poly_arrays(default_L)
# print(U)

# print("***********")

V = convert_arrays_into_poly_arrays(default_R)
# print(V)

# print("***********")

W = convert_arrays_into_poly_arrays(default_S)
# print(W)

# print("***********")

###############################################

Ua = multiply_poly_arrays_with_witness(U)
Va = multiply_poly_arrays_with_witness(V)
Wa = multiply_poly_arrays_with_witness(W)

# print(Ua)
# print("***********")
# print(Va)
# print("***********")
# print(Wa)

###############################################
## Introducing t(x) and h(x)

def negateGF967(num):
    return GF967(0) - GF967(num)

a = galois.Poly(Ua, GF967)
b = galois.Poly(Va, GF967)
c = galois.Poly(Wa, GF967)
t = galois.Poly(GF967(np.array([1, negateGF967(1)])), GF967) * galois.Poly(GF967(np.array([1, negateGF967(2)])), GF967) * galois.Poly(GF967(np.array([1, negateGF967(3)])), GF967)

print("a: ", a)
print("b: ", b)
print("c: ", c)
print("t: ", t)

h = ((a * b) - c ) // t

print("h: ", h) 

a:  513x^2 + 396x + 61
b:  966x^2 + 4x
c:  568x^2 + 237x + 171
t:  x^3 + 961x^2 + 11x + 961
h:  454x + 512


In [5]:
from py_ecc.bn128 import G1, G2, pairing, multiply, add, eq
import random

x = 53.5123
print(math.modf(53.5123))
print(math.modf(x))
print(math.modf(x)[0] == 0.0)
print(math.modf(x)[0] * 100)
x = math.modf(x)[0] * 100
print(int(x))

example_1 = -6.5 % 11
print(example_1) # 3.5
# Since, 3.5 == 35 x 10**-1
print((35%11 * pow(10, -1, 11))%11)

example_2 = (65%11 * pow(10, -1, 11))%11
print(-example_2 % 11)

## example 3
## -6 in GF (the above defined galios field)
print(GF967(0) - GF967(6))
print(GF967(0) - GF967(1))

## example 4
## 5/2 in GF (the above defined galios field)
print(GF967(0) - (GF967(5) * GF967(2)**-1))

############################################################################################################
## Introducing tau
## Assume this is generated in a trusted setup and both the prover and verifier are unaware of it's value

tau = random.choice(GF967.elements)
print("Tau: ", tau)

## Non-encrypted evaluation

LHS = a(tau) * b(tau)
print(LHS)

RHS = c(tau) + h(tau) * t(tau)
print(RHS)

if(LHS == RHS):
    print("Congratulations!!")
else:
    print("Lmeow, fuck you, you cheat!!")

######################################

print(tau)

tau0 = tau ** 0
tau1 = tau ** 1
tau2 = tau ** 2

tau_0_g1 = multiply(G1, int(tau0)) 
tau_1_g1 = multiply(G1, int(tau1))
tau_2_g1 = multiply(G1, int(tau2))

tau_0_g2 = multiply(G2, int(tau0)) 
tau_1_g2 = multiply(G2, int(tau1))
tau_2_g2 = multiply(G2, int(tau2))

encrypted_g1_points = [tau_2_g1, tau_1_g1, tau_0_g1]
encrypted_g2_points = [tau_2_g2, tau_1_g2, tau_0_g2]

###########################################################
## Evaluating polynomial `a` at encrypted tau

print(a.coeffs)

a_1 = multiply(encrypted_g1_points[0], int(a.coeffs[0]))
a_2 = multiply(encrypted_g1_points[1], int(a.coeffs[1]))
a_3 = multiply(encrypted_g1_points[2], int(a.coeffs[2]))

a_final = add(a_1, add(a_2, a_3))

print(a_final)

###########################################################
## Evaluating polynomial `b` at encrypted tau

# print(b.coeffs)

b_1 = multiply(encrypted_g2_points[0], int(b.coeffs[0]))
b_2 = multiply(encrypted_g2_points[1], int(b.coeffs[1]))
b_3 = multiply(encrypted_g2_points[2], int(b.coeffs[2]))

b_final = add(b_1, add(b_2, b_3))
# b_final = add(b_1, b_2)

print(b_final)

###########################################################
## Evaluating polynomial `c` at encrypted tau

# print(c.coeffs)

c_1 = multiply(encrypted_g1_points[0], int(c.coeffs[0]))
c_2 = multiply(encrypted_g1_points[1], int(c.coeffs[1]))
c_3 = multiply(encrypted_g1_points[2], int(c.coeffs[2]))

c_final = add(c_1, add(c_2, c_3))

print("c_final: ", c_final)

############################################################
## Evaluating polynomial h(x)t(x) at encrypted tau

print(t.coeffs)
print(h.coeffs)

t_at_tau = t(tau)
t_1_mul = GF967(tau1) * GF967(t_at_tau)
t_2_mul = GF967(tau0) * GF967(t_at_tau)

t_1 = multiply(G1, int(t_1_mul))
t_2 = multiply(G1, int(t_2_mul))

ht_1 = multiply(t_1, int(h.coeffs[0]))
ht_2 = multiply(t_2, int(h.coeffs[1]))

ht_final = add(ht_1, ht_2)

print("t(tau): ", t_at_tau)
print("ht_final: ", ht_final)

###############################################################
## Final evaluation: Pairing the two points on either sides

c_dash = add(c_final, ht_final)

print("c' : ", c_dash)

lhs = pairing(b_final, a_final)
rhs = pairing(G2, c_dash)

##################################################################################
## This comparison should fail because we are using G1 and G2 points
## from the bn128 curve which is applied over a prime field of bn128.curve_order
## and our calculation upto now was done in the prime field of 967, ie GF967
##################################################################################

if(eq(rhs, lhs)):
    print("Yayy!! The proof was correct")
else:
    print("Nope, invalid proof")


(0.5123000000000033, 53.0)
(0.5123000000000033, 53.0)
False
51.23000000000033
51
4.5
9
10
961
966
481
Tau:  336
886
886
Congratulations!!
336
[513 396  61]
(14466953843806189672678717816021537543036611669525827354577272951422906980424, 5011239353952754949850690819031377233476891796752857446921356467837397653245)
((679682102959526337548475907738370199009464426632529025506574257870130779077, 12214085684685938590642250011642486343930658826257643971232826659898787510956), (5387265637471795450612786999968664419491334849397341699655149511997643493569, 3383094241684455934796207170073457092048110178460464524890886241342535561597))
c_final:  (8739104360331627103762341063792162827868773637493948543199474279528724318158, 13254927435344648534740265507693102812972215351922275708811840769714980500967)
[  1 961  11 961]
[454 512]
t(tau):  860
ht_final:  (21170896267442177819489317952212633490222767568756989413445121854654507005730, 172090148349939264349997858942895998165892066835707737229913758695313